# 03 — Análisis de Cartera: métricas y Monte Carlo

In [ ]:

import sys
from pathlib import Path
base = Path.cwd(); src = base / "src"
if str(src) not in sys.path: sys.path.insert(0, str(src))
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
sns.set_theme(context="notebook", style="whitegrid")
from mcport import PriceSeries, Portfolio, MonteCarloSimulation, MonteCarloPlots, var_cvar

idx = pd.bdate_range('2022-01-03','2024-12-31')
def gbm(n, mu, sigma, s0, seed):
    rng=np.random.default_rng(seed); dt=1/252
    z=rng.standard_normal(n-1)
    path=np.r_[0, np.cumsum((mu-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*z)]
    return s0*np.exp(path)
a = PriceSeries(symbol="A", asset_type="equity", currency="USD", provider="sim",
                data=pd.DataFrame({"price":gbm(len(idx),0.10,0.20,100,1)}, index=idx))
b = PriceSeries(symbol="B", asset_type="equity", currency="USD", provider="sim",
                data=pd.DataFrame({"price":gbm(len(idx),0.06,0.10, 80,2)}, index=idx))
c = PriceSeries(symbol="C", asset_type="equity", currency="USD", provider="sim",
                data=pd.DataFrame({"price":gbm(len(idx),0.14,0.30, 50,3)}, index=idx))
w = [0.5, 0.3, 0.2]
port = Portfolio(positions=[a,b,c], weights=w, name="Demo_3_assets", currency="USD")

eq = port.value_series(1.0)
r = np.log(eq).diff().dropna()
mu, sigma = r.mean(), r.std(ddof=1)
v95, c95 = var_cvar(r, alpha=0.05)
print("μ=", mu, "σ=", sigma, "VaR95=", v95, "CVaR95=", c95)

mc = MonteCarloSimulation(price_series=port)
sims = mc.monte_carlo(days=252, n_sims=500, seed=42)
MonteCarloPlots(mc).plot_history_with_simulations(sims)
plt.show()
